In [ ]:
import pandas as pd

properties_csv = pd.read_csv("/Users/thughes/tmp/properties_2016.csv", low_memory=False)
train_csv = pd.read_csv("/Users/thughes/tmp/train_2016_v2.csv", low_memory=False)

In [ ]:
from datetime import datetime

train_csv['month'] = train_csv['transactiondate'].map(lambda x: datetime.strptime(x, '%Y-%m-%d').month)

In [ ]:
for mon in range (1, 13):
    train_csv['2016'+str(mon)] = train_csv[train_csv['month'] == mon]['logerror']


In [ ]:
train_csv = train_csv.drop(['logerror', 'month'], axis=1)

In [ ]:
train_csv = train_csv.groupby('parcelid').first()

In [ ]:
train_csv = train_csv.drop("transactiondate", axis=1)

In [ ]:
df = properties_csv.join(train_csv, on="parcelid")

In [ ]:
df = pd.get_dummies(df, columns=["hashottuborspa", "propertycountylandusecode", "propertyzoningdesc", 
              "fireplaceflag", "taxdelinquencyflag"])


In [ ]:
df10 = df.dropna(subset=['201610'])
df11 = df.dropna(subset=['201611'])
df12 = df.dropna(subset=['201612'])

In [ ]:
from sklearn.model_selection import train_test_split

X10_train, X10_test, y10_train, y10_test = train_test_split(df10.drop(['201610', '201611', '201612'], axis=1), df10['201610'], test_size=0.2)
X11_train, X11_test, y11_train, y11_test = train_test_split(df11.drop(['201611', '201612'], axis=1), df10['201611'], test_size=0.2)
X12_train, X12_test, y12_train, y12_test = train_test_split(df12.drop(['201612'], axis=1), df12['201612'], test_size=0.2)

In [ ]:
from xgboost import XGBRegressor

reg10 = XGBRegressor()
reg11 = XGBRegressor()
reg12 = XGBRegressor()

reg10.fit(X10_train, y10_train)
reg11.fit(X11_train, y11_train)
reg12.fit(X12_train, y12_train)

In [ ]:
preds10 = reg10.predict(df)
preds11 = reg11.predict(df)
preds12 = reg12.predict(df)

In [ ]:
from sklearn.metrics import mean_squared_error

#mean_squared_error(preds, y_test)

In [ ]:
final = pd.DataFrame()
final['parcelid'] = properties_csv['parcelid']
final['201610'] = preds10
final['201710'] = preds10
final['201611'] = preds11
final['201711'] = preds11
final['201712'] = preds12
final['201612'] = preds12

In [ ]:
final.write_csv("submit.csv", header=True)